# Introduction to magic methods by actual use case

How polynomials are being __added__ by numpy?

In [ ]:
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt

A = Polynomial(coef=[0, 0, 1], domain=[-2, 2], window=[-2, 2])  # 0 + 0*x + 1*x^2
B = Polynomial(coef=[1, 0, 0], domain=[-2, 2], window=[-2, 2])  # 1 + 0*x + 0*x^2
C = A + B  # (1 + 0*x + 1*x^2)

plt.plot(*A.linspace(), label=f"A ({A})", linestyle="-", color="green")
plt.plot(*B.linspace(), label=f"B ({B})", linestyle="--", color="red")
plt.plot(*C.linspace(), label=f"C ({C})", linestyle="-.", color="orange")

plt.legend()
plt.axvline(0)
plt.axhline(0)
plt.show()

Let's see how dataframes could be __updated__ by Pandas?

In [ ]:
import pandas as pd

prices = [10, 20, 30]
df = pd.DataFrame({"prices": prices})

print(f'Initial {df}')

df *= 1.05
print(f'Updated {df}')

Conclusions: 
* both __numpy__ and __Pandas__ deliver implementations of methods whose name start and end with a double underscore
* implementations could be easily found with breakpoints.

# First implementation: Operator overloading and string representation
Let's assume:
$$v1 = (10, 20)$$
$$v2 = (50, 60)$$

Our objectives:
* provide method responsible for creating a resultant vector $$ v3 = v1 + v2 $$
* provide method responsible for vector's string representation <code>str(v3)</code>
* provide method responsible for calculating a product of a vector and a number $$ v4 = v3 * n $$



In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2

The addition failed as Python doesn't know how to add two <code>Vector</code> objects.

We'll add our first magic method <code>\_\_add__()</code> that is called then Python spots the <code>+</code> operator.

In [ ]:
   def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

Now the updated code does not trigger any exceptions.

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2
print(v3)

But we still can't see the result.

## The string representations

We'll add a magic method <code>\_\_str__()</code> to get a string representation of an object. Most common use case: when you print the object.

In [ ]:
    def __str__(self):
        return f'Vector({self.x}, {self.y})'

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __str__(self):
        return f'Vector({self.x}, {self.y})'

v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2
print(v3)

The <code>\_\_repr__</code> method provides the official string representation of an object, aimed at the developer.

The <code>\_\_str__</code> method provides the informal string representation of an object, aimed at the user.




In [ ]:
    def __repr__(self):
        return f'Vector [x: {self.x}, y:{self.y}]'

## Operator overloading

The <code>\_\_mul__()</code> method defines a behavior for what's returned when you invoke multiplication on an object. 

## Exercise #1: calculate a product of a vector and a number

Your task is to implement the <code>\_\_mul()__</code> method that creates a new vector, that is the product of a vector and a number. 

Hints: 
* you could use the following formula:
$$v = (a, b)$$
$$v * n = (a * n, b * n)$$
* the prepared assertion will validate the result  


In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __str__(self):
        return f'Vector({self.x}, {self.y})'

    def __mul__(self, number):
        return ...  # <---- put your code here
        
v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2
v4 = v3 * 3
assert str(v4) == str(Vector(180, 240))

## Exercise #2: validate data types

Magic methods should validate input parameters. In case the input data type is incorrect, magic method should return singleton <code>__NotImplemented__</code> value or raise appropriate exception. 

Your task is to extend the <code>\_\_mul()__</code> method to validate the <code>number</code> parameter type. Only integers and floats could be used to multiply the vector; the use of any other data types should be rejected by returning the <code>NotImplemented</code> value.

## Right-hand operators

When you try to execute the following code:

In [ ]:
v41 = 3 * v3
assert str(v41) == str(Vector(180, 240))


You'll receive the <code>TypeError</code>, because magic methods that support operators are affected by the relative position of each object in the containing expression.

__Solution__:
Provide the right-hand version of operator methods <code>\_\_r*\_\_()</code>, eg.<code>\_\_rmul\_\_()</code>. 


## Exercise #3: unary operators

Python supports unary arithmetic operations (-, +, abs() and ~). The unary operator (-) is supported by the <code>\_\_neg__</code> method. 

Your task is to extend the <code>Vector</code> class by creating an implementation of the method that returns the opposite vector.


In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __str__(self):
        return f'Vector({self.x}, {self.y})'

    def __mul__(self, number):
        if isinstance(number, float | int):
            return Vector(self.x * number, self.y * number)
        else:
            return NotImplemented

    def __rmul__(self, number):
         return self.__mul__(number)        

    def __neg__(self):
        return ...  # <---- put your code here


v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2

v4 = v3 * 3
assert str(v4) == str(Vector(180, 240))

v41 = 3 * v3
assert str(v41) == str(Vector(180, 240))

v5 = -v4
assert str(v5) == str(Vector(-180, -240))


## Exercise #4: compare two vectors

Python offers special methods that are called for comparison operators. For example, when you want to test if object are equal (by value, but not identity) with equal sign <code>=</code>, Python calls the <code>\_\_eq__()</code> magic method.

Your task is to extend the <code>Vector</code> class by creating an implementation of the forementioned <code>\_\_eq__()</code> method.

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __str__(self):
        return f'Vector({self.x}, {self.y})'

    def __mul__(self, number):
        if isinstance(number, float | int):
            return Vector(self.x * number, self.y * number)
        else:
            return NotImplemented

    def __rmul__(self, number):
         return self.__mul__(number)        

    def __neg__(self):
        return Vector(-self.x, -self.y)

    def __eq__(self, other):
        return ...  # <---- put your code here

v1 = Vector(10, 20)
v2 = Vector(50, 60)

v3 = v1 + v2
v4 = v3 * 3
assert str(v4) == str(Vector(180, 240))

v4 = 3 * v3
assert str(v4) == str(Vector(180, 240))

v5 = -v4
assert str(v5) == str(Vector(-180, -240))

assert v5 == Vector(-180, -240)